<a href="https://colab.research.google.com/github/YaninaK/churn-prediction/blob/main/notebooks/03_Inference_pipeline_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Прогнозирование оттока клиентов
## Конвейер для инференса нейросетевой модели.

## [EDA и отбор признаков](https://github.com/YaninaK/churn-prediction/blob/main/notebooks/01_EDA_and_Feature_selection.ipynb)

## [Базовая нейросетевая модель](https://github.com/YaninaK/churn-prediction/blob/main/notebooks/02_Baseline_model_NN.ipynb)

[Ссылка на данные](https://drive.google.com/file/d/1TAVECAfnel9lPfcpfel6qXhZSW2yNqdX/view?usp=sharing)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/YaninaK/churn-prediction.git -q

In [3]:
%cd churn-prediction

/content/churn-prediction


In [4]:
import sys
import os

sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), "src", "churn_prediction"))

In [5]:
import logging
import pandas as pd
import numpy as np
import easydict

from models.inference_tools import preprocessing_pipeline
from models.serialize import load

In [6]:
PATH = '/content/drive/MyDrive/ML_projects/06_Churn_prediction/'

## Конвейер инференса.

Файл c конвейером инференса для запуска из командной строки [по ссылке](https://github.com/YaninaK/churn-prediction/blob/main/scripts/inference_script.py)

In [7]:
args = easydict.EasyDict(
    {
        'data_path': PATH + 'data/01_raw/data_test.csv',
        'artifacts_path': PATH,
        'model_name': 'LSTM_emb_model_v1',
        'output': 'data/06_model_output/scores.csv',
    }
)

In [8]:
def main(args):
    logging.info("Reading data...")

    data = pd.read_csv(args.data_path)

    logging.info("Preprocessing data...")

    test_features, test_labels = preprocessing_pipeline(data, args.artifacts_path)

    logging.info("Loading the model...")

    lstm_model = load(args.model_name, args.artifacts_path + 'models/')

    logging.info("Performing inference...")

    predictions = lstm_model.predict(test_features)

    logging.info("Calculating clients scores...")

    result = calculate_scores(predictions, test_features)

    logging.info("Saving scores...")

    result.to_csv(args.output)


def calculate_scores(predictions, test_features):
    result = (predictions * 1000 // 10)
    result = pd.DataFrame(result, index=test_features[-1].index, columns=['scores'])
    result['scores'] = result['scores'].astype(int)

    return result


In [9]:
main(args)

59/59 [==============================] - 1s 1ms/step


## Чтение результатов

In [10]:
df = pd.read_csv('data/06_model_output/scores.csv')
df

,customerid,scores
0,2,10
1,4,59
2,5,28
3,6,0
4,9,70
...,...,...
1862,8645,87
1863,8646,13
1864,8649,23
1865,8651,63
